## Book 3: Analyzing Neighborhoods in Toronto, Canada using Foursquare

Peer-graded assignment for IBM Data Science Professional certification capstone course: Week 3 - Brian Vineyard

**Tools used**:
- [BeautifulSoup](https://https://beautiful-soup-4.readthedocs.io/en/latest/)
- [Foursquare API tool](https://foursquare.com)
- [http://cocl.us/Geospatial_data](http://cocl.us/Geospatial_data)

Toronto neighborhoods information is loaded using postal codes pulled from Wikipedia using BeautifulSoup, then analyzed using the Foursquare API tool.

This project uses the following Wiki page: [List of postal codes of Canada: M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) 

### Steps to analyze Toronto Neighborhood Data

1. Load the wiki containing postal codes for Toronto beginning with the letter 'M' into a Pandas dataframe. BeautifulSoup is used to scrape the wiki for the postal data. 
2. The data is cleaned up for processing.
3. Load the Geospatial data from a .CSV file built from [http://cocl.us/Geospatial_data](http://cocl.us/Geospatial_data). This provides latitude and longitude parameters to use in Foursquare.
4. Build out the dataframe with the Toronto neighborhoods including the map coordinates.
5. Analyze and model the data to gain insights into neighborhood characteristics.

**This notebook includes the prior work for phase 1 and 2, and new steps for phase 3, analyzing Toronto neighborhoods using the Foursquare API.


## <font color="green"> Phase 1 - Build pandas dataframe using BeautifulSoup</font>  ##

In [3]:
# Import Pandas and Beautiful Soup libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Link to source data and parsing of Wiki website using BeautifulSoup

In [4]:
# Use Beautiful Soup to get the postal code data from the wiki page
sourcelink = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(sourcelink).text
Postal_Info = BeautifulSoup(source, 'lxml')

### Set up Pandas dataframe to hold postal data

In [5]:
# Build dataframe with postal data
Column_Names = ['PostalCode','Borough','Neighborhood']
Postal_Data = pd.DataFrame(columns=Column_Names)

### Initialize variables for postal code, borough, and neighborhood, then loop through table rows to pull data

In [6]:
content = Postal_Info.find('div', class_='mw-parser-output')
postal_table = content.table.tbody
postal_code = 0
borough = 0
neighborhood = 0

for tr in postal_table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postal_code = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2:
            neighborhood = td.text.strip('\n').replace(']','')
            Postal_Data = Postal_Data.append({'Postalcode': postal_code,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [7]:
Postal_Data.head()

,PostalCode,Borough,Neighborhood,Postalcode
0,NaN,Not assigned,Not assigned,M1A
1,NaN,Not assigned,Not assigned,M2A
2,NaN,North York,Parkwoods,M3A
3,NaN,North York,Victoria Village,M4A
4,NaN,Downtown Toronto,Harbourfront,M5A


## Data cleaning - remove boroughs with 'Not assigned',  0 values

In [8]:
Postal_Data = Postal_Data[Postal_Data.Borough!='Not assigned']
Postal_Data = Postal_Data[Postal_Data.Borough!= 0]
Postal_Data.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,Postal_Data.shape[0]):
    if Postal_Data.iloc[i][2] == 'Not assigned':
        Postal_Data.iloc[i][2] = Postal_Data.iloc[i][1]
        i = i+1
df = Postal_Data.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [9]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]   

### Get first 5 and last 5 rows of data from the postal dataframe

In [10]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df.tail()

,Postalcode,Borough,Neighborhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


### Group data by Postal Code and Borough

In [12]:
def neighborhood_list(grouped):
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [13]:
print(df2.shape)
df2.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## <font color="green"> Phase 2 - Add map coordinates for Toronto neighborhoods based on Geospatial data</font>  ##

### Load Geospatial data from .csv file built on data from http://cocl.us/Geospatial_data ###

In [14]:
df3 = pd.read_csv('C:/users/brian/coordinates.csv', delimiter="|")
df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Join two dataframes into combined set including coordinates, then view first 5 rows ###

In [15]:
neighborhood_coords = [df2, df3]
neighborhood_df = pd.concat(neighborhood_coords, axis=1)

In [16]:
neighborhood_df.head()

,Postalcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


### Remove the duplicate 'Postal Code' column, and view head of combined dataframe ###

In [17]:
del neighborhood_df['Postal Code']

In [18]:
neighborhood_df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Build map of Toronto, Canada showing the neighborhoods from the dataframe ###

In [31]:
print('The dataframe has {} boroughs and {} neighborhoods.' .format(len(neighborhood_df['Borough'].unique()), neighborhood_df.shape[0]))

The dataframe has 11 boroughs and 103 neighborhoods.


In [33]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.653963, -79.387207.


In [36]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhood_df['Latitude'], neighborhood_df['Longitude'], neighborhood_df['Borough'], neighborhood_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
    
map_toronto    

## <font color="green"> Phase 3 - Analyze Toronto neighborhoods using the Foursquare API</font>  ##

### Import libraries ###

In [37]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

from geopy.geocoders import Nominatim 
import requests # library to handle requests
from pandas.io.json import json_normalize
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
print('Libraries imported.')

Libraries imported.


### Load Foursquare parameters ###

In [41]:
file = open('credentials.p', 'rb')
credentials = pickle.load(file)
file.close()
CLIENT_ID = credentials['CLIENT_ID']
CLIENT_SECRET = credentials['CLIENT_SECRET']
VERSION = '20180604'
LIMIT = 30

### Convert address info for the Doubletree Hotel in downtown Toronto to latitude and longitude coordinates ###

In [42]:
address = '108 Chestnut St, Toronto, ON'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6546682 -79.38609661430215


### Set query paramaters to search for restaurants close by the hotel ###

In [43]:
search_query = 'Restaurant'
radius = 500
print(search_query + ' .....OK!')

Restaurant .....OK!


In [44]:
# Set up the search url to query Foursquare
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [45]:
# Get the results as a JSON file
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e290698edbcad001b806141'},
 'response': {'venues': [{'id': '4ad4c05ff964a52048f720e3',
    'name': 'Hemispheres Restaurant & Bistro',
    'location': {'address': '110 Chestnut Street',
     'lat': 43.65488413420439,
     'lng': -79.38593077371578,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65488413420439,
       'lng': -79.38593077371578}],
     'distance': 27,
     'postalCode': 'M5G 1R3',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['110 Chestnut Street',
      'Toronto ON M5G 1R3',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'shortName': 'American',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1579746988',
    'hasPerk': False},
   {'id': '4

In [46]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",False,4ad4c05ff964a52048f720e3,110 Chestnut Street,CA,Toronto,Canada,NaN,27,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'label': 'display', 'lat': 43.65488413420439...",43.654884,-79.385931,M5G 1R3,ON,Hemispheres Restaurant & Bistro,v-1579746988,NaN
1,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,4ae4b055f964a520229d21e3,143 Dundas St. West,CA,Toronto,Canada,NaN,103,"[143 Dundas St. West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65538110598594...",43.655381,-79.385270,NaN,ON,Kyoto House Japanese Restaurant,v-1579746988,NaN
2,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,52a7ae41498eed3af4d0a3fa,126 Elizabeth St.,CA,Toronto,Canada,Dundas St.,91,"[126 Elizabeth St. (Dundas St.), Toronto ON, C...","[{'label': 'display', 'lat': 43.65528126342919...",43.655281,-79.385337,NaN,ON,Yueh Tung Chinese Restaurant,v-1579746988,NaN
3,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4b2027b5f964a520f82d24e3,195 Dundas St W,CA,Toronto,Canada,at University Ave,84,"[195 Dundas St W (at University Ave), Toronto ...","[{'label': 'display', 'lat': 43.65492521335936...",43.654925,-79.387089,M5G 1C7,ON,Hong Shing Chinese Restaurant,v-1579746988,60327598
4,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4b295e10f964a520ba9d24e3,14 Queen St W,CA,Toronto,Canada,NaN,524,"[14 Queen St W, Toronto ON M5H 3X4, Canada]","[{'label': 'display', 'lat': 43.65261436174172...",43.652614,-79.380231,M5H 3X4,ON,Richtree Natural Market Restaurants,v-1579746988,NaN


In [47]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,CA,Toronto,Canada,NaN,27,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'label': 'display', 'lat': 43.65488413420439...",43.654884,-79.385931,M5G 1R3,ON,4ad4c05ff964a52048f720e3
1,Kyoto House Japanese Restaurant,Sushi Restaurant,143 Dundas St. West,CA,Toronto,Canada,NaN,103,"[143 Dundas St. West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65538110598594...",43.655381,-79.385270,NaN,ON,4ae4b055f964a520229d21e3
2,Yueh Tung Chinese Restaurant,Chinese Restaurant,126 Elizabeth St.,CA,Toronto,Canada,Dundas St.,91,"[126 Elizabeth St. (Dundas St.), Toronto ON, C...","[{'label': 'display', 'lat': 43.65528126342919...",43.655281,-79.385337,NaN,ON,52a7ae41498eed3af4d0a3fa
3,Hong Shing Chinese Restaurant,Chinese Restaurant,195 Dundas St W,CA,Toronto,Canada,at University Ave,84,"[195 Dundas St W (at University Ave), Toronto ...","[{'label': 'display', 'lat': 43.65492521335936...",43.654925,-79.387089,M5G 1C7,ON,4b2027b5f964a520f82d24e3
4,Richtree Natural Market Restaurants,Restaurant,14 Queen St W,CA,Toronto,Canada,NaN,524,"[14 Queen St W, Toronto ON M5H 3X4, Canada]","[{'label': 'display', 'lat': 43.65261436174172...",43.652614,-79.380231,M5H 3X4,ON,4b295e10f964a520ba9d24e3
5,Cali Restaurant,Vietnamese Restaurant,179 Dundas St. W.,CA,Toronto,Canada,at Chestnut,49,"[179 Dundas St. W. (at Chestnut), Toronto ON M...","[{'label': 'display', 'lat': 43.65506808, 'lng...",43.655068,-79.386375,M5G,ON,4c476d6719fde21e32410876
6,Wah Too Seafood Restaurant,Chinese Restaurant,56 Centre Ave.,CA,Toronto,Canada,NaN,91,"[56 Centre Ave., Toronto ON M5G 1R5, Canada]","[{'label': 'display', 'lat': 43.65483285234745...",43.654833,-79.387206,M5G 1R5,ON,4c69740b8d22c9284d42b745
7,Tundra Restaurant,Restaurant,145 Richmond Street West,CA,Toronto,Canada,Hilton Toronto,520,"[145 Richmond Street West (Hilton Toronto), To...","[{'label': 'display', 'lat': 43.65000998764964...",43.650010,-79.385608,M5H 2L2,ON,50ca02c0245f2d4aa8c2b313
8,New Treasure Restaurant,Dim Sum Restaurant,150 Dundas St W,CA,Toronto,Canada,at Elizabeth,99,"[150 Dundas St W (at Elizabeth), Toronto ON, C...","[{'label': 'display', 'lat': 43.65538444237565...",43.655384,-79.385362,NaN,ON,4b2a674ef964a52074a824e3
9,Hendricks Restaurant & Bar,Restaurant,Yonge And Queen,CA,Toronto,Canada,NaN,533,"[Yonge And Queen, Toronto ON M5G 2H6, Canada]","[{'label': 'display', 'lat': 43.65341542752959...",43.653415,-79.379698,M5G 2H6,ON,5a8b24b2018cbb6c4ca70dae


In [48]:
dataframe_filtered.name

0                   Hemispheres Restaurant & Bistro
1                   Kyoto House Japanese Restaurant
2                      Yueh Tung Chinese Restaurant
3                     Hong Shing Chinese Restaurant
4               Richtree Natural Market Restaurants
5                                   Cali Restaurant
6                        Wah Too Seafood Restaurant
7                                 Tundra Restaurant
8                           New Treasure Restaurant
9                        Hendricks Restaurant & Bar
10                                 Adega Restaurant
11    Spring Rolls | Japanese Restaurant in Toronto
12                            Restaurant | Adelaide
13                       Alio Restaurant & Wine Bar
14                          The Elm Tree Restaurant
15               Akashiro Japanese Restaurant & Bar
16                                     Lai Wah Heen
17                          Little India Restaurant
18                          Osgoode Hall Restaurant
19          

### Map of downtown Toronto, showing restaurants within a 500 meter radius of the Doubletree Hotel

In [49]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Doubletree Hotel

# add a red circle marker to represent the Doubletree Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Doubletree Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Look at data for the Hemispheres restaurant ###

In [52]:
venue_id = '4ad4c05ff964a52048f720e3' # ID of Hemispheres Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

In [53]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto', 'colors'])


{'id': '4ad4c05ff964a52048f720e3',
 'name': 'Hemispheres Restaurant & Bistro',
 'contact': {'phone': '4165998000',
  'formattedPhone': '(416) 599-8000',
  'twitter': 'methotels'},
 'location': {'address': '110 Chestnut Street',
  'lat': 43.65488413420439,
  'lng': -79.38593077371578,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.65488413420439,
    'lng': -79.38593077371578}],
  'postalCode': 'M5G 1R3',
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['110 Chestnut Street', 'Toronto ON M5G 1R3', 'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/hemispheres-restaurant--bistro/4ad4c05ff964a52048f720e3',
 'categories': [{'id': '4bf58dd8d48988d14e941735',
   'name': 'American Restaurant',
   'pluralName': 'American Restaurants',
   'shortName': 'American',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
    'suffix': '.png'},
   'primary': True}],
 'verified': True,
 'stats': {'tipCount': 5},
 'ur

### Rating data for the Hemispheres restaurant ###

In [54]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.7


### Look at data for the Yueh Tang restaurant ###

In [76]:
venue_id = '52a7ae41498eed3af4d0a3fa' # ID of Yueh Tang Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

In [77]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '52a7ae41498eed3af4d0a3fa',
 'name': 'Yueh Tung Chinese Restaurant',
 'contact': {'phone': '4169770933', 'formattedPhone': '(416) 977-0933'},
 'location': {'address': '126 Elizabeth St.',
  'crossStreet': 'Dundas St.',
  'lat': 43.655281263429195,
  'lng': -79.3853365267765,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.655281263429195,
    'lng': -79.3853365267765}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['126 Elizabeth St. (Dundas St.)',
   'Toronto ON',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/yueh-tung-chinese-restaurant/52a7ae41498eed3af4d0a3fa',
 'categories': [{'id': '4bf58dd8d48988d145941735',
   'name': 'Chinese Restaurant',
   'pluralName': 'Chinese Restaurants',
   'shortName': 'Chinese',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 10},
 'url': 'http://www.yuehtungr

### Rating data for the Yueh Tang restaurant ###

In [78]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.9


### Get tips for Yueh Tang ###

In [57]:
result['response']['venue']['tips']['count']

5

In [58]:
## The Yueh Tang Restaurant Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2906c6fb34b5001bcde9eb'},
 'response': {'tips': {'count': 10,
   'items': [{'id': '581bb07fd67c728bae67294f',
     'createdAt': 1478209663,
     'text': '30th October 2016 -YUEH TUNG CHINESE RESTAURANT VERY TASTY  FOOD -I LOVED THE CHILLI BEEF & CHILLI FISH YUM YUM👅',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/581bb07fd67c728bae67294f',
     'photo': {'id': '581bb07fbda2d73777fafdd2',
      'createdAt': 1478209663,
      'source': {'name': 'Foursquare for Android',
       'url': 'https://foursquare.com/download/#/android'},
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'suffix': '/159607071_1YLasoAidR7phN5vJv912EVohyInxFAXfvQfTVDs1U8.jpg',
      'width': 1032,
      'height': 581,
      'visibility': 'public'},
     'photourl': 'https://fastly.4sqi.net/img/general/original/159607071_1YLasoAidR7phN5vJv912EVohyInxFAXfvQfTVDs1U8.jpg',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logVie

In [59]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [79]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,30th October 2016 -YUEH TUNG CHINESE RESTAURANT VERY TASTY FOOD -I LOVED THE CHILLI BEEF & CHILLI FISH YUM YUM👅,0,0,581bb07fd67c728bae67294f,Linette,G,NaN,159607071


### Explore area around the Yueh Tang Restaurant location ###

In [80]:
# Latitude and Longitude of Yueh Tang
latitude = 43.655281
longitude = -79.385337

In [81]:
# Pass url including location data to Foursquare
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

In [82]:
# Load results into JSON file, then print number of points of interest
results = requests.get(url).json()
'There are {} points of interest around the Yueh Tang restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 points of interest around the Yueh Tang restaurant.'

In [83]:
# Build items list from JSON results
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5227bb01498e17bf485e6202',
  'name': 'Downtown Toronto',
  'location': {'lat': 43.65323167517444,
   'lng': -79.38529600606677,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.65323167517444,
     'lng': -79.38529600606677}],
   'distance': 228,
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['Toronto ON', 'Canada']},
  'categories': [{'id': '4f2a25ac4b909258e854f55f',
    'name': 'Neighborhood',
    'pluralName': 'Neighborhoods',
    'shortName': 'Neighborhood',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-5227bb01498e17bf485e6202-0'}

In [84]:
# Build dataframe with normalized JSON data, filtering columns and categories, and cleaning up column headings
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Downtown Toronto,Neighborhood,NaN,CA,Toronto,Canada,NaN,228,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65323167517444, 'lng': -79.38529600606677}]",43.653232,-79.385296,NaN,NaN,ON,5227bb01498e17bf485e6202
1,Red Lobster,Seafood Restaurant,20 Dundas St W,CA,Toronto,Canada,at Bay St,180,"[20 Dundas St W (at Bay St), Toronto ON M5G 2C2, Canada]","[{'label': 'display', 'lat': 43.656328, 'lng': -79.383621}]",43.656328,-79.383621,NaN,M5G 2C2,ON,4ae3398ff964a520ed9121e3
2,Japango,Sushi Restaurant,122 Elizabeth St.,CA,Toronto,Canada,at Dundas St. W,13,"[122 Elizabeth St. (at Dundas St. W), Toronto ON M5G 1P5, Canada]","[{'label': 'display', 'lat': 43.65526771691681, 'lng': -79.38516506734886}]",43.655268,-79.385165,NaN,M5G 1P5,ON,4ae7b27df964a52068ad21e3
3,Chatime 日出茶太,Bubble Tea Shop,132 Dundas St W,CA,Toronto,Canada,btwn Bay & University,60,"[132 Dundas St W (btwn Bay & University), Toronto ON M5G 1C3, Canada]","[{'label': 'display', 'lat': 43.65554164147378, 'lng': -79.38468427043244}]",43.655542,-79.384684,NaN,M5G 1C3,ON,4e2284b11fc7c0ef9857d143
4,MUJI,Miscellaneous Shop,595 Bay St E,CA,Toronto,Canada,at Dundas St W,184,"[595 Bay St E (at Dundas St W), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.656024, 'lng': -79.383284}]",43.656024,-79.383284,NaN,NaN,ON,5479da4f498e8569fb44985c
5,Rolltation,Japanese Restaurant,207 Dundas St W,CA,Toronto,Canada,at University Ave,172,"[207 Dundas St W (at University Ave), Toronto ON M5G 1C8, Canada]","[{'label': 'display', 'lat': 43.65491791857301, 'lng': -79.3874242454196}]",43.654918,-79.387424,NaN,M5G 1C8,ON,5773f01f498e98371390bdfd
6,Tsujiri,Tea Room,147 Dundas St W,CA,Toronto,Canada,at Elizabeth St,10,"[147 Dundas St W (at Elizabeth St), Toronto ON M5G 1P5, Canada]","[{'label': 'display', 'lat': 43.65537430780922, 'lng': -79.38535434742991}]",43.655374,-79.385354,NaN,M5G 1P5,ON,56ccd5cfcd1069ca160a797e
7,Sansotei Ramen 三草亭,Ramen Restaurant,179 Dundas St. W,CA,Toronto,Canada,btwn Centre Ave. & Chestnut St.,94,"[179 Dundas St. W (btwn Centre Ave. & Chestnut St.), Toronto ON M5G 1Z8, Canada]","[{'label': 'display', 'lat': 43.655157467561246, 'lng': -79.38650067479335}]",43.655157,-79.386501,NaN,M5G 1Z8,ON,504bbf2ce4b0168121235cbe
8,Poke Guys,Poke Place,112 Elizabeth St,CA,Toronto,Canada,at Dundas St W,48,"[112 Elizabeth St (at Dundas St W), Toronto ON M5G 1P5, Canada]","[{'label': 'display', 'lat': 43.65489527525682, 'lng': -79.38505238381624}]",43.654895,-79.385052,NaN,M5G 1P5,ON,57bcd3b7498e652a678d0378
9,The Elm Tree Restaurant,Modern European Restaurant,43 Elm St,CA,Toronto,Canada,Bay,267,"[43 Elm St (Bay), Toronto ON M5G 1H1, Canada]","[{'label': 'display', 'lat': 43.65739749535259, 'lng': -79.38376054171513}]",43.657397,-79.383761,NaN,M5G 1H1,ON,539c6f13498e06f4cc765165


### Build and display map using Folium to show points of interest close to the Yueh Tang restaurant ###

In [85]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Yueh Tang Chinese Restaurant',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

### Get trending venues, if any are found from the area around the Yueh Tang ###

In [69]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e290719b9a389001bb364a7'},
 'response': {'venues': [{'id': '4b155081f964a520b4b023e3',
    'name': 'Scotiabank Arena',
    'location': {'address': '40 Bay St',
     'lat': 43.64344617535107,
     'lng': -79.37903988889991,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.64344617535107,
       'lng': -79.37903988889991}],
     'distance': 1411,
     'postalCode': 'M5J 2X2',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['40 Bay St', 'Toronto ON M5J 2X2', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d18b941735',
      'name': 'Basketball Stadium',
      'pluralName': 'Basketball Stadiums',
      'shortName': 'Basketball',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/stadium_basketball_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'id': '33431410'}}]}}

In [70]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

### Display results and area map of trending venues query, with message if no values are found ###

In [71]:
# display trending venues
trending_venues_df

,name,categories,location.distance,location.city,location.postalCode,location.state,location.country,location.lat,location.lng
0,Scotiabank Arena,Basketball Stadium,1411,Toronto,M5J 2X2,ON,Canada,43.643446,-79.37904


In [72]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [93]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'